In [ ]:
import os
from pathlib import Path
from collections import defaultdict

**Data Loading**

In [ ]:
# Set the path to your Desktop dataset folder
base_path = Path.home() / "Desktop" / "fdmproject"

# Adjust folder names as needed
datasets = {
    "Pneumonia": base_path / "PneumoniaDataset",
    "TB": base_path / "TB_split_dataset"
}

def count_images(dataset_path):
    counts = defaultdict(dict)
    for split in ['train', 'val', 'test']:
        split_path = dataset_path / split
        if not split_path.exists():
            continue
        for cls in os.listdir(split_path):
            cls_path = split_path / cls
            if not cls_path.is_dir(): 
                continue
            num_images = len([
                f for f in os.listdir(cls_path)
                if os.path.isfile(cls_path / f) and f.lower().endswith(('.jpg', '.jpeg', '.png'))
            ])
            counts[split][cls] = num_images
    return counts

# Report counts
for name, path in datasets.items():
    print(f"\n📊 {name} Dataset")
    stats = count_images(path)
    for split, classes in stats.items():
        print(f"  {split.upper()}:")
        for cls, count in classes.items():
            print(f"    {cls:<10}: {count} images")

In [ ]:
import os
import shutil
import random
from glob import glob
from sklearn.model_selection import train_test_split
from PIL import Image
import cv2
import albumentations as A

In [ ]:
BASE_PATH = os.path.expanduser("~/Desktop/fdmproject")
RAW_DATA = os.path.join(BASE_PATH, "TBDataset")  # Contains "Normal" and "TB"
OUTPUT_PATH = os.path.join(BASE_PATH, "TB_split_dataset")

tb_images = glob(os.path.join(RAW_DATA, "TB", "*"))
normal_images = glob(os.path.join(RAW_DATA, "Normal", "*"))

tb_train, tb_temp = train_test_split(tb_images, test_size=0.25, random_state=42)
tb_val, tb_test = train_test_split(tb_temp, test_size=0.6, random_state=42)

normal_train, normal_temp = train_test_split(normal_images, test_size=0.25, random_state=42)
normal_val, normal_test = train_test_split(normal_temp, test_size=0.6, random_state=42)

for split in ["train", "val", "test"]:
    for cls in ["TB", "Normal"]:
        os.makedirs(os.path.join(OUTPUT_PATH, split, cls), exist_ok=True)
        
def copy_images(image_paths, dest_dir):
    for path in image_paths:
        shutil.copy(path, os.path.join(dest_dir, os.path.basename(path)))
        
copy_images(tb_train, os.path.join(OUTPUT_PATH, "train", "TB"))
copy_images(tb_val, os.path.join(OUTPUT_PATH, "val", "TB"))
copy_images(tb_test, os.path.join(OUTPUT_PATH, "test", "TB"))

copy_images(normal_val, os.path.join(OUTPUT_PATH, "val", "Normal"))
copy_images(normal_test, os.path.join(OUTPUT_PATH, "test", "Normal"))

num_tb_train = len(tb_train)
num_normal_train = len(normal_train)
target_normal = num_tb_train
images_to_generate = target_normal - num_normal_train

augmentor = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.7),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=10, p=0.5),
    A.GaussianBlur(blur_limit=3, p=0.2)
])

normal_output_dir = os.path.join(OUTPUT_PATH, "train", "Normal")
i = 0
while i < images_to_generate:
    img_path = random.choice(normal_train)
    img = cv2.imread(img_path)
    if img is None:
        continue
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    augmented = augmentor(image=img)["image"]
    aug_img = Image.fromarray(augmented)
    aug_img.save(os.path.join(normal_output_dir, f"aug_normal_{i}.jpg"))
    i += 1
    
copy_images(normal_train, normal_output_dir)

print("✅ TB_split_dataset created with 75/10/15 split and balanced training set.")

**Data Augmentation and Preprocessing**

In [ ]:
import os
from pathlib import Path
import random

In [ ]:
train_normal_dir = Path.home() / "Desktop" / "fdmproject" / "PneumoniaDataset" / "train" / "NORMAL"

In [ ]:
# Collect only augmented images (starting with 'aug_')
augmented_images = sorted([
    f for f in os.listdir(train_normal_dir)
    if f.startswith('aug_') and f.lower().endswith(('.jpg', '.jpeg', '.png'))
])

In [ ]:
# Sanity check
print(f"Found {len(augmented_images)} augmented images.")

In [ ]:
# Choose 600 to delete (randomly)
num_to_delete = 600
to_delete = random.sample(augmented_images, min(num_to_delete, len(augmented_images)))

In [ ]:
# Delete them
for fname in to_delete:
    path = train_normal_dir / fname
    os.remove(path)

In [ ]:
print(f"Deleted {len(to_delete)} augmented images from: {train_normal_dir}")

In [ ]:
import os
import random
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
# Set base folder (adjust if needed)
base_path = Path.home() / "Desktop" / "fdmproject"

In [ ]:
# Define class folders
classes = {
    "Pneumonia - NORMAL": base_path / "PneumoniaDataset" / "train" / "NORMAL",
    "Pneumonia - PNEUMONIA": base_path / "PneumoniaDataset" / "train" / "PNEUMONIA",
    "TB - Normal": base_path / "TB_split_dataset" / "train" / "Normal",
    "TB - TB": base_path / "TB_split_dataset" / "train" / "TB"
}

In [ ]:
# Plot samples per class
samples_per_class = 3

In [ ]:
fig, axs = plt.subplots(len(classes), samples_per_class, figsize=(samples_per_class * 2, len(classes) * 2.5))
fig.suptitle("📊 Sample Images from Each Class", fontsize=16)

In [ ]:
for row, (label, path) in enumerate(classes.items()):
    images = [f for f in os.listdir(path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    chosen = random.sample(images, min(samples_per_class, len(images)))

In [ ]:
for col, fname in enumerate(chosen):
        img_path = path / fname
        img = Image.open(img_path).convert('L')  # Convert to grayscale
        axs[row, col].imshow(img, cmap='gray')
        axs[row, col].axis('off')
        if col == 0:
            axs[row, col].set_title(label, loc='left', fontsize=10)

In [ ]:
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

In [ ]:
import cv2
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms

In [ ]:
# Sample image path (use any image from your dataset)
img_path = Path.home() / "Desktop" / "fdmproject" / "PneumoniaDataset" / "train" / "NORMAL"
sample_file = sorted(img_path.glob("*.jpeg"))[0]  # Choose first .jpeg

In [ ]:
# Load original
original = Image.open(sample_file).convert('L')
original_np = np.array(original)

In [ ]:
# Apply preprocessing
equalized = cv2.equalizeHist(original_np)
resized = cv2.resize(equalized, (224, 224))
normalized = (resized / 255.0 - 0.5) / 0.5  # Simulate torchvision.Normalize([0.5], [0.5])

In [ ]:
# Plot
fig, axs = plt.subplots(1, 3, figsize=(12, 4))
axs[0].imshow(original_np, cmap='gray')
axs[0].set_title("Original")
axs[1].imshow(equalized, cmap='gray')
axs[1].set_title("Histogram Equalized")
axs[2].imshow(resized, cmap='gray')
axs[2].set_title("Resized to 224x224")

In [ ]:
for ax in axs: ax.axis('off')
plt.suptitle("🧪 Image Preprocessing Steps", fontsize=14)
plt.tight_layout()
plt.show()

**Model Training- Pneumonia**

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm

In [ ]:
BASE_PATH = os.path.expanduser("~/Desktop/fdmproject")
PNEUMONIA_PATH = os.path.join(BASE_PATH, "PneumoniaDataset")
TB_PATH = os.path.join(BASE_PATH, "TB_split_dataset")
IMG_SIZE = 224
BATCH_SIZE = 32
NUM_EPOCHS = 6

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

def get_loaders(base):
    return (
        DataLoader(datasets.ImageFolder(os.path.join(base, 'train'), transform), batch_size=BATCH_SIZE, shuffle=True),
        DataLoader(datasets.ImageFolder(os.path.join(base, 'val'), transform), batch_size=BATCH_SIZE),
        DataLoader(datasets.ImageFolder(os.path.join(base, 'test'), transform), batch_size=BATCH_SIZE),
        datasets.ImageFolder(os.path.join(base, 'train'), transform).classes
    )
    
p_train, p_val, p_test, p_classes = get_loaders(PNEUMONIA_PATH)
t_train, t_val, t_test, t_classes = get_loaders(TB_PATH)

def train_pneumonia():
    model = models.efficientnet_b0(pretrained=True)
    for p in model.parameters():
        p.requires_grad = False
        
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)
    model.to(device)
    
opt = optim.Adam(model.parameters(), lr=1e-4)
    loss_fn = nn.CrossEntropyLoss()
    best_acc = 0

In [ ]:
for epoch in range(NUM_EPOCHS):
        model.train()
        total, correct = 0, 0
        for x, y in tqdm(p_train, desc=f"[Pneumonia] Epoch {epoch+1}/{NUM_EPOCHS}"):
            x, y = x.to(device), y.to(device)
            out = model(x)
            loss = loss_fn(out, y)
            opt.zero_grad(); loss.backward(); opt.step()
            correct += (torch.argmax(out, 1) == y).sum().item()
            total += y.size(0)
            
torch.save(model.state_dict(), os.path.join(BASE_PATH, "best_model_pneumonia.pth"))
    print("\n✅ Pneumonia model training complete. Best model saved.")
    
print("\n📊 Pneumonia Validation Evaluation:")
    evaluate(model, p_val, p_classes)
    
print("\n📊 Pneumonia Test Evaluation:")
    evaluate(model, p_test, p_classes)

In [ ]:
class GradReverse(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, lambda_):
        ctx.lambda_ = lambda_
        return x.view_as(x)
    
@staticmethod
    def backward(ctx, grad_output):
        return grad_output.neg() * ctx.lambda_, None

def grad_reverse(x, lambda_):
    return GradReverse.apply(x, lambda_)

class DANN_EfficientNet(nn.Module):
    def __init__(self):
        super().__init__()
        base = models.efficientnet_b0(pretrained=False)
        self.feature_extractor = nn.Sequential(*list(base.children())[:-1])
        self.classifier = nn.Linear(base.classifier[1].in_features, 2)
        self.domain_classifier = nn.Sequential(
            nn.Linear(base.classifier[1].in_features, 100),
            nn.ReLU(),
            nn.Linear(100, 2)
        )

for name, param in self.feature_extractor.named_parameters():
            if any(f"features.{i}" in name for i in [3, 4, 5, 6, 7]):
                param.requires_grad = True
            else:
                param.requires_grad = False
                
def forward(self, x, lambda_=0.0):
        features = self.feature_extractor(x).squeeze()
        class_out = self.classifier(features)
        domain_out = self.domain_classifier(grad_reverse(features, lambda_))
        return class_out, domain_out

**Model Training-TB with DANN**

In [ ]:
def train_dann():
    model = DANN_EfficientNet().to(device)
    pretrained = torch.load(os.path.join(BASE_PATH, "best_model_pneumonia.pth"), map_location=device)
    model.load_state_dict({k: v for k, v in pretrained.items() if k in model.state_dict() and v.size() == model.state_dict()[k].size()}, strict=False)
    
opt = optim.Adam(model.parameters(), lr=1e-4)
    loss_class = nn.CrossEntropyLoss()
    loss_domain = nn.CrossEntropyLoss()
    
for epoch in range(NUM_EPOCHS):
        model.train()
        total_loss = 0
        zipped = zip(t_train, p_train)
        for (x_t, y_t), (x_s, _) in tqdm(zipped, desc=f"[TB DANN] Epoch {epoch+1}/{NUM_EPOCHS}"):
            x_t, y_t = x_t.to(device), y_t.to(device)
            x_s = x_s.to(device)
            
x_all = torch.cat([x_t, x_s], dim=0)
            domain_labels = torch.cat([
                torch.ones(x_t.size(0)),
                torch.zeros(x_s.size(0))
            ]).long().to(device)
            
            
class_out, domain_out = model(x_all, lambda_=0.1)
            loss = loss_class(class_out[:x_t.size(0)], y_t) + loss_domain(domain_out, domain_labels)
            
opt.zero_grad(); loss.backward(); opt.step()
            total_loss += loss.item()
            
print(f"Epoch {epoch+1}: DANN Loss = {total_loss:.4f}")

torch.save(model.state_dict(), os.path.join(BASE_PATH, "dann_tb.pth"))
    print("DANN fine-tuned TB model saved as dann_tb.pth")
    return model

**Evaluation**

In [ ]:
def evaluate(model, loader, classes, return_acc=False):
    model.eval()
    all_preds, all_labels, all_probs = [], [], []
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            out = model(x)[0] if isinstance(model(x), tuple) else model(x)
            probs = torch.softmax(out, dim=1)
            preds = (probs[:, 1] > 0.6).long()  # Threshold tuning: TB only if >60% confident

In [ ]:
all_probs.extend(probs[:, 1].cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y.cpu().numpy())

In [ ]:
print(classification_report(all_labels, all_preds, target_names=classes))
    print(confusion_matrix(all_labels, all_preds))

In [ ]:
# ROC Curve
    fpr, tpr, _ = roc_curve(all_labels, all_probs)
    roc_auc = auc(fpr, tpr)
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
if return_acc:
        return 100 * (np.array(all_preds) == np.array(all_labels).astype(int)).sum() / len(all_labels)

In [ ]:
if __name__ == '__main__':
    
    print("\n📌 Step 2: Fine-tuning on TB with DANN")
    model = train_dann()
    
    print("\n📊 TB Validation Evaluation:")
    evaluate(model, t_val, t_classes)
    
    print("\n📊 TB Test Evaluation:")
    evaluate(model, t_test, t_classes)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
# Plot
plt.figure(figsize=(6, 5))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues",
            xticklabels=class_names, yticklabels=class_names)

plt.xlabel("Predicted Label", fontsize=12)
plt.ylabel("True Label", fontsize=12)
plt.title("Confusion Matrix", fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
import os, time, torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from tqdm import tqdm
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import numpy as np
import matplotlib.pyplot as plt

**Model Training and Evaluation: TB Final**

In [ ]:
BASE_PATH = os.path.expanduser("~/Desktop/fdmproject")
TB_PATH = os.path.join(BASE_PATH, "TB_split_dataset")
PNEUMO_MODEL_PATH = os.path.join(BASE_PATH, "best_model_pneumonia.pth")
IMG_SIZE = 224
BATCH_SIZE = 32
NUM_EPOCHS = 6

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

def get_loaders(base):
    return (
        DataLoader(datasets.ImageFolder(os.path.join(base, 'train'), transform), batch_size=BATCH_SIZE, shuffle=True),
        DataLoader(datasets.ImageFolder(os.path.join(base, 'val'), transform), batch_size=BATCH_SIZE),
        DataLoader(datasets.ImageFolder(os.path.join(base, 'test'), transform), batch_size=BATCH_SIZE),
        datasets.ImageFolder(os.path.join(base, 'train'), transform).classes
    )
    
t_train, t_val, t_test, t_classes = get_loaders(TB_PATH)

class SimpleEfficientNet(nn.Module):
    def __init__(self):
        super().__init__()
        base = models.efficientnet_b0(pretrained=True)
        self.feature_extractor = nn.Sequential(*list(base.children())[:-1])
        self.classifier = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(base.classifier[1].in_features, 2)
        )

for name, param in self.feature_extractor.named_parameters():
            if any(f"features.{i}" in name for i in [4, 5, 6, 7]):
                param.requires_grad = True
            else:
                param.requires_grad = False
                
def forward(self, x):
        features = self.feature_extractor(x).squeeze()
        return self.classifier(features)

In [ ]:
# ───────────────────────────────
# EVALUATION
# ───────────────────────────────
def evaluate(model, loader, classes, return_acc=False, silent=False):
    model.eval()
    all_preds, all_labels, all_probs = [], [], []
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            probs = torch.softmax(out, dim=1)
            preds = (probs[:, 1] > 0.45).long()  # Lower threshold to improve Normal recall
            all_probs.extend(probs[:, 1].cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y.cpu().numpy())

In [ ]:
if not silent:
        print(classification_report(all_labels, all_preds, target_names=classes))
        print(confusion_matrix(all_labels, all_preds))
        fpr, tpr, _ = roc_curve(all_labels, all_probs)
        roc_auc = auc(fpr, tpr)
        plt.figure()
        plt.plot(fpr, tpr, label=f'AUC = {roc_auc:.2f}')
        plt.plot([0, 1], [0, 1], '--')
        plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate')
        plt.title('ROC Curve'); plt.legend(); plt.show()

In [ ]:
if return_acc:
        return 100 * (np.array(all_preds) == np.array(all_labels)).sum() / len(all_labels)

In [ ]:
# ───────────────────────────────
# TRAINING
# ───────────────────────────────
def train_transfer_model():
    class EarlyStopper:
        def __init__(self, patience=2):
            self.patience = patience
            self.counter = 0
            self.best_acc = 0

In [ ]:
def check(self, val_acc):
            if val_acc > self.best_acc:
                self.best_acc = val_acc
                self.counter = 0
                return False
            else:
                self.counter += 1
                return self.counter >= self.patience

In [ ]:
model = SimpleEfficientNet().to(device)

In [ ]:
# Load pneumonia model weights
    pretrained = torch.load(PNEUMO_MODEL_PATH, map_location=device)
    model.load_state_dict({k: v for k, v in pretrained.items() if k in model.state_dict() and v.size() == model.state_dict()[k].size()}, strict=False)

In [ ]:
opt = optim.Adam(model.parameters(), lr=5e-5, weight_decay=1e-4)
    loss_fn = nn.CrossEntropyLoss(label_smoothing=0.1, weight=torch.tensor([1.1, 1.0]).to(device))
    early_stopper = EarlyStopper(patience=2)

In [ ]:
for epoch in range(NUM_EPOCHS):
        model.train()
        total_loss = 0
        start_time = time.time()

In [ ]:
for x, y in tqdm(t_train, desc=f"[Transfer Learning] Epoch {epoch+1}/{NUM_EPOCHS}"):
            x, y = x.to(device), y.to(device)
            out = model(x)
            loss = loss_fn(out, y)

In [ ]:
opt.zero_grad(); loss.backward(); opt.step()
            total_loss += loss.item()

In [ ]:
elapsed = time.time() - start_time
        print(f"Epoch {epoch+1}: Loss = {total_loss:.4f} | Time: {elapsed:.2f} sec")

In [ ]:
val_acc = evaluate(model, t_val, t_classes, return_acc=True, silent=True)
        print(f"Validation Accuracy: {val_acc:.2f}%")
        if early_stopper.check(val_acc):
            print(f"🛑 Early stopping triggered at epoch {epoch+1}")
            break

In [ ]:
torch.save(model.state_dict(), os.path.join(BASE_PATH, "transfer_tb_final.pth"))
    print("✅ Final Transfer model saved as transfer_tb_final.pth")
    return model

In [ ]:
# ───────────────────────────────
# RUN
# ───────────────────────────────
model = train_transfer_model()
print("\n📊 Final TB Validation Evaluation:")
evaluate(model, t_val, t_classes, silent=False)

In [ ]:
print("\n📊 Final TB Test Evaluation:")
evaluate(model, t_test, t_classes, silent=False)

In [ ]:
from torchvision import models

In [ ]:
# Rebuild model class
class SimpleEfficientNet(nn.Module):
    def __init__(self):
        super().__init__()
        base = models.efficientnet_b0(pretrained=False)
        self.feature_extractor = nn.Sequential(*list(base.children())[:-1])
        self.classifier = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(base.classifier[1].in_features, 2)
        )

In [ ]:
def forward(self, x):
        features = self.feature_extractor(x).squeeze()
        return self.classifier(features)

In [ ]:
# Load model
model = SimpleEfficientNet()
model.load_state_dict(torch.load(os.path.join(BASE_PATH, "transfer_tb.pth")))
model.to(device)

In [ ]:
# Freeze all except classifier
for name, param in model.named_parameters():
    param.requires_grad = False
for param in model.classifier.parameters():
    param.requires_grad = True

In [ ]:
# Setup fine-tune
optimizer = optim.Adam(model.classifier.parameters(), lr=5e-5)
loss_fn = nn.CrossEntropyLoss(weight=torch.tensor([1.2, 1.0]).to(device))

In [ ]:
# Data loader for fine-tuning
fine_tune_loader = DataLoader(
    datasets.ImageFolder(os.path.join(TB_PATH, 'train'), transform),
    batch_size=32, shuffle=True
)

In [ ]:
# Train for 2 short epochs
model.train()
for epoch in range(2):
    total_loss = 0
    for x, y in tqdm(fine_tune_loader, desc=f"🛠 Fine-tuning Epoch {epoch+1}/2"):
        x, y = x.to(device), y.to(device)
        out = model(x)
        loss = loss_fn(out, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1} Loss: {total_loss:.4f}")

In [ ]:
# Save fine-tuned version
torch.save(model.state_dict(), os.path.join(BASE_PATH, "transfer_tb_normalboost.pth"))
print("✅ Saved: transfer_tb_normalboost.pth")

In [ ]:
# Rebuild the same model structure
class SimpleEfficientNet(nn.Module):
    def __init__(self):
        super().__init__()
        base = models.efficientnet_b0(pretrained=False)
        self.feature_extractor = nn.Sequential(*list(base.children())[:-1])
        self.classifier = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(base.classifier[1].in_features, 2)
        )

In [ ]:
def forward(self, x):
        features = self.feature_extractor(x).squeeze()
        return self.classifier(features)

In [ ]:
# Load and evaluate
model = SimpleEfficientNet()
model.load_state_dict(torch.load(os.path.join(BASE_PATH, "transfer_tb_normalboost.pth")))
model.to(device)
model.eval()

In [ ]:
print("\n📊 Final Evaluation After Normal Recall Boost:")
evaluate(model, t_test, t_classes, silent=False)

In [ ]:
import os
import random
import cv2
import matplotlib.pyplot as plt
import albumentations as A

In [ ]:
# Paths
BASE_PATH = os.path.expanduser("~/Desktop/fdmproject")
PNEUMO_PATH = os.path.join(BASE_PATH, "PneumoniaDataset", "train", "NORMAL")
TB_PATH = os.path.join(BASE_PATH, "TB_split_dataset", "train", "Normal")

In [ ]:
# Presentation-safe augmentations
demo_augmentor = A.Compose([
    A.HorizontalFlip(p=1.0),
    A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=1.0),
    A.GaussianBlur(blur_limit=3, p=1.0),
])

In [ ]:
def load_and_augment(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    aug = demo_augmentor(image=image)["image"]
    return image, aug

In [ ]:
# Select sample images
pneu_img_path = os.path.join(PNEUMO_PATH, random.choice(os.listdir(PNEUMO_PATH)))
tb_img_path = os.path.join(TB_PATH, random.choice(os.listdir(TB_PATH)))

In [ ]:
pneu_original, pneu_aug = load_and_augment(pneu_img_path)
tb_original, tb_aug = load_and_augment(tb_img_path)

In [ ]:
# Plot
fig, axs = plt.subplots(2, 2, figsize=(10, 6))
axs[0, 0].imshow(pneu_original); axs[0, 0].set_title("Pneumonia NORMAL - Original")
axs[0, 1].imshow(pneu_aug); axs[0, 1].set_title("Pneumonia NORMAL - Augmented")
axs[1, 0].imshow(tb_original); axs[1, 0].set_title("TB NORMAL - Original")
axs[1, 1].imshow(tb_aug); axs[1, 1].set_title("TB NORMAL - Augmented")

In [ ]:
for ax in axs.flatten():
    ax.axis("off")

In [ ]:
plt.tight_layout()
plt.show()